In [1]:
from rdflib import *
from rdflib import URIRef

In [2]:
spoonR = Graph()

In [3]:
spoonR.parse("/Users/anita/Desktop/ERAS/ontology/kg.ttl", format="ttl")

<Graph identifier=N7fa2d2ad6e53445c9400ca2037a5b08e (<class 'rdflib.graph.Graph'>)>

In [6]:
#What are the topics covered in each album? How do topics evolve over the years? #works
q1
=
"""SELECT ?album (GROUP_CONCAT(DISTINCT ?topic; SEPARATOR=", ") AS ?topics) ?releaseYear
WHERE {
  ?album rdf:type :Album .
  ?album :hasTopic ?topic .
  ?album :has_release_year ?releaseYear .
}
GROUP BY ?album ?releaseYear
ORDER BY ?releaseYear
"""


#How frequently do these topics appear across the albums? #works
q2= """
SELECT ?topic (COUNT(?album) AS ?count)
WHERE {
  ?album rdf:type :Album .
  ?album :hasTopic ?topic .
}
GROUP BY ?topic
ORDER BY ?topic
    """


#What topics are related to the different genres?  #works
q3= """
SELECT ?genre (GROUP_CONCAT(DISTINCT ?topic; SEPARATOR=", ") AS ?topics)
WHERE {
  ?album rdf:type :Album .
  ?album :hasGenre ?genre .
  ?album :hasTopic ?topic .
}
GROUP BY ?genre
ORDER BY ?genre
    """

#Which is the distribution of album across the genres? #words
q4="""
SELECT ?genre (COUNT(?album) AS ?count)
WHERE {
  ?album rdf:type :Album .
  ?album :has_release_year ?year .
  ?album :hasGenre ?genre .
}
GROUP BY ?genre
ORDER BY ?genre
"""

#Which topics are associated with the labels? #works
q5="""
SELECT ?label ?topic
WHERE {
  { SELECT ?label ?topic
    WHERE {
      ?album rdf:type :Album .
      ?album :belongsToLabel ?label .
      ?album :hasTopic ?topic .
      FILTER (?label = :Big_Machine)
    }
  }
  UNION
  { SELECT ?label ?topic
    WHERE {
      ?album rdf:type :Album .
      ?album :belongsToLabel ?label .
      ?album :hasTopic ?topic .
      FILTER (?label = :Republic)
    }
  }
  UNION
  { SELECT ?label ?topic
    WHERE {
      ?album rdf:type :Album .
      ?album :belongsToLabel ?label .
      ?album :hasTopic ?topic .
      FILTER (?label = :Taylor_Swift_Productions)
    }
  }
}

"""


#LOVER AND REPUTATION  #works
q6 = """
SELECT ?albumTitle ?topic ?genre
WHERE {
  {
    ?album rdf:type :Album .
    ?album :has_title ?albumTitle .
    OPTIONAL { ?album :hasTopic ?topic }
    OPTIONAL { ?album :hasGenre ?genre }
    FILTER (str(?albumTitle) = "reputation")
  }
  UNION
  {
    ?album rdf:type :Album .
    ?album :has_title ?albumTitle .
    OPTIONAL { ?album :hasTopic ?topic }
    OPTIONAL { ?album :hasGenre ?genre }
    FILTER (str(?albumTitle) = "Lover")
  }
  
  # Check for shared topics between the two albums
  FILTER EXISTS {
    ?otherAlbum rdf:type :Album .
    ?otherAlbum :has_title ?otherAlbumTitle .
    ?otherAlbum :hasTopic ?topic .
    FILTER (?otherAlbumTitle != ?albumTitle)
  }
  
  # Check for shared genres between the two albums
  FILTER EXISTS {
    ?otherAlbum rdf:type :Album .
    ?otherAlbum :has_title ?otherAlbumTitle .
    ?otherAlbum :hasGenre ?genre .
    FILTER (?otherAlbumTitle != ?albumTitle)
  }
}
"""


#FOKLORE AND EVERMORE #works
q7 = """
SELECT ?albumTitle ?topic ?genre
WHERE {
  {
    ?album rdf:type :Album .
    ?album :has_title ?albumTitle .
    OPTIONAL { ?album :hasTopic ?topic }
    OPTIONAL { ?album :hasGenre ?genre }
    FILTER (str(?albumTitle) = "folkore")
  }
  UNION
  {
    ?album rdf:type :Album .
    ?album :has_title ?albumTitle .
    OPTIONAL { ?album :hasTopic ?topic }
    OPTIONAL { ?album :hasGenre ?genre }
    FILTER (str(?albumTitle) = "evermore")
  }
  
  # Check for shared topics between the two albums
  FILTER EXISTS {
    ?otherAlbum rdf:type :Album .
    ?otherAlbum :has_title ?otherAlbumTitle .
    ?otherAlbum :hasTopic ?topic .
    FILTER (?otherAlbumTitle != ?albumTitle)
  }
  
  # Check for shared genres between the two albums
  FILTER EXISTS {
    ?otherAlbum rdf:type :Album .
    ?otherAlbum :has_title ?otherAlbumTitle .
    ?otherAlbum :hasGenre ?genre .
    FILTER (?otherAlbumTitle != ?albumTitle)
  }
} 
"""


#Longest album #works
q8="""
SELECT ?albumTitle ?duration
WHERE {
  ?album rdf:type :Album .
  ?album :has_title ?albumTitle .
  ?album :has_duration ?duration .
}
ORDER BY DESC(?duration)
LIMIT 1"""

#shortest album #works

q9="""
SELECT ?albumTitle ?duration
WHERE {
  ?album rdf:type :Album .
  ?album :has_title ?albumTitle .
  ?album :has_duration ?duration .
}
ORDER BY ASC(?duration)
LIMIT 1

"""


#album that shares topics #works
q10 = """
SELECT ?album1 ?album2 (GROUP_CONCAT(DISTINCT ?topic; SEPARATOR=", ") AS ?sharedTopics)
WHERE {
  ?album1 rdf:type :Album .
  ?album1 :hasTopic ?topic .
  ?album2 rdf:type :Album .
  ?album2 :hasTopic ?topic .
  FILTER (?album1 != ?album2)
}
GROUP BY ?album1 ?album2
HAVING (COUNT(DISTINCT ?topic) > 0)
ORDER BY ?album1 ?album2
"""

#shared genres #works
q11 = """
SELECT ?album1 ?album2 (GROUP_CONCAT(DISTINCT ?genre; SEPARATOR=", ") AS ?sharedGenres)
WHERE {
  ?album1 rdf:type :Album .
  ?album1 :hasGenre ?genre .
  ?album2 rdf:type :Album .
  ?album2 :hasGenre ?genre .
  FILTER (?album1 != ?album2)
}
GROUP BY ?album1 ?album2
ORDER BY ?album1 ?album2

"""


#which songs are about reflection? #not working
q12= """
SELECT ?song ?album ?topic
WHERE {
  ?song rdf:type :Song .
  ?song :hasTopic :reflection .
  OPTIONAL { ?song :belongsToAlbum ?album }
  BIND(:reflection AS ?topic)
}
"""


#which songs shares topics?
q13= """
SELECT ?song1 ?song2 (GROUP_CONCAT(DISTINCT ?topic; SEPARATOR=", ") AS ?sharedTopics)
WHERE {
  ?song1 rdf:type :Song .
  ?song1 :hasTopic ?topic .
  ?song2 rdf:type :Song .
  ?song2 :hasTopic ?topic .
  FILTER (?song1 != ?song2)
}
GROUP BY ?song1 ?song2
HAVING (COUNT(DISTINCT ?topic) > 0)
ORDER BY ?song1 ?song2
"""

#which is the song featured by Lana Del Rey and which is the topic?
q14= """
SELECT ?song ?topic ?album
WHERE {
  ?song rdf:type :Song .
  ?song :hasFeaturedArtist :Lana_Del_Rey .
  OPTIONAL { ?song :hasTopic ?topic }
  OPTIONAL { ?song :belongsToAlbum ?album }
}
"""

q15= """
    SELECT ?songTitle ?featuredArtist
    WHERE {
    ?song rdf:type :Song .
    ?song :has_title ?songTitle .
    ?song :hasFeaturedArtist ?featuredArtist .
    }
    """

queries = [q1, q2, q3, q4, q5, q6, q7, q8, q9, q10, q11, q12, q13, q14, q15]

def get_local_name(uri):
    return uri.split('#')[-1] if '#' in uri else uri.split('/')[-1]

# Function to extract local name from a URIRef
for idx, q in enumerate(queries):
    print("Results for query " + str(idx + 1) + ":")
    try:
        results = spoonR.query(q)
        if results:
            for result in results:
                if idx in [0, 5, 6, 9, 10, 11, 12, 13]:  # Query 8 (index 7)
                    album, topic, genre = result
                    album_local = get_local_name(str(album))
                    topic_local = get_local_name(str(topic))
                    genre_local = get_local_name(str(genre))
                    print(f"result: {album_local}, {topic_local}, {genre_local}")
                else:
                    album, topic = result
                    album_local = get_local_name(str(album))
                    topic_local = get_local_name(str(topic))
                    print(f"result: {album_local}, result: {topic_local}")
        else:
            print("No results found.")
    except Exception as e:
        print(f"An error occurred: {e}")

Results for query 1:
result: Debut, nostalgia, 2006
result: Fearless, uncertainty, 2008
result: Speak_Now, return_and_departure, 2010
result: Red, regret, 2012
result: 1989, turmoil, 2014
result: Reputation, reflection, 2017
result: Lover, relationships, 2019
result: Evermore, suffering, 2020
result: Folklore, struggle, 2020
result: Midnights, seeking_understanding, 2022
result: The_Tortured_Poets_Department, turmoil, 2024
Results for query 2:
result: adventure, result: 1
result: anger, result: 1
result: appreciation, result: 1
result: aspirations, result: 1
result: being_emotional, result: 2
result: breakup, result: 1
result: challenges, result: 1
result: companionship, result: 1
result: conflict, result: 1
result: decision-making, result: 1
result: desire, result: 4
result: desperation, result: 1
result: devotion, result: 1
result: discovery, result: 1
result: enjoying_life, result: 1
result: enthusiasm, result: 1
result: excitement, result: 1
result: expression, result: 2
result: fa